## Uploading Files from Kaggle

In [1]:
! pip install -q kaggle
from google.colab import files



In [2]:
# choose the kaggle.json file that you downloaded
files.upload()
! mkdir ~/.kaggle


Saving kaggle.json to kaggle.json


In [4]:

# make a directory named kaggle and copy the kaggle.json file there
!cp kaggle.json ~/.kaggle/



In [5]:
# change the permissions of the file
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d subhankarpanda56/news-bias

 86% 94.0M/110M [00:01<00:00, 51.2MB/s]
100% 110M/110M [00:01<00:00, 69.8MB/s] 


In [7]:
! unzip news-bias.zip

Archive:  news-bias.zip
  inflating: new_bias.csv            


## Installing and importing Libraries

In [8]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.6 MB/s eta 0:00:00


In [9]:
import transformers

from transformers import TFRobertaForSequenceClassification
from transformers import RobertaTokenizer


import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')


import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
df = pd.read_csv("/content/new_bias.csv")
df.head(5)

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,terrorism,New York Times - News,0,http://www.nytimes.com/2016/09/20/nyregion/ahm...,"Bomb Suspect Changed After Trip Abroad, Friend...",2016-09-20,N. R. Kleinfield,"Besides his most recent trip to Quetta , Mr. R...","Besides his most recent trip to Quetta, Mr. Ra...",www.nytimes.com,left,004Gt3gcsotuiYmz
1,supreme_court,Vox,0,https://www.vox.com/policy-and-politics/2018/9...,Why Susan Collins claims she’s being bribed ov...,2018-09-12,"Emily Stewart, Terry Nguyen, Rebecca Jennings,...",Is Maine Republican Sen. Susan Collins being b...,Is Maine Republican Sen. Susan Collins being b...,www.vox.com,left,00eP4XD3VdMmHITE
2,education,Ezra Klein,0,http://www.npr.org/blogs/thetwo-way/2014/05/06...,Poll: Prestigious Colleges Won't Make You Happ...,2014-05-06,Anya Kamenetz,Poll : Prestigious Colleges Wo n't Make You Ha...,Poll: Prestigious Colleges Won't Make You Happ...,www.npr.org,left,00FTGIZEd6B8zQ4U
3,us_house,Breitbart News,2,http://www.breitbart.com/big-government/2017/0...,Paul Ryan Reportedly Says No Chance for Border...,2017-09-12,Ian Mason,"House Speaker Paul Ryan , at a private dinner ...","House Speaker Paul Ryan, at a private dinner e...",www.breitbart.com,right,00HGGqBRf1kzPRlg
4,white_house,Guest Writer - Left,0,https://www.cnn.com/2019/07/11/politics/donald...,OPINION: Trump seeking change of legal fortune...,2019-07-11,Analysis Stephen Collinson,( CNN ) President Donald Trump has reason to h...,(CNN) President Donald Trump has reason to hop...,www.cnn.com,left,00IzI5ynahBVtC9l


In [11]:
import numpy as np
np.random.seed(42)

# Randomly select 12,000 rows from the DataFrame
df1 = df.sample(n=6000)


In [12]:
texts = df1['content'].to_list()
labels = df1['bias'].to_list()

In [13]:
df1.bias.value_counts()

2    2189
0    2059
1    1752
Name: bias, dtype: int64

## Preprocessing

In [14]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size = 0.2, random_state = 0 )


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )

In [15]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [16]:
train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

test_encodings =  tokenizer(test_texts, truncation = True, padding = True )

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))



In [18]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
! pip install "ray[tune]"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 35.3 MB/s eta 0:00:00


## Training

In [29]:
from transformers import TFRobertaForSequenceClassification, TFTrainer, TFTrainingArguments


training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=2,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    eval_steps=100,
    fp16 = True,
    evaluation_strategy = "steps",
    learning_rate = 3e-5
)

with training_args.strategy.scope():
    trainer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)


trainer = TFTrainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/trainer_tf.py:118: FutureWarning:

The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow



In [30]:
trainer.train()

## Testing

In [31]:
trainer.evaluate()

{'eval_loss': 0.5535908749229029}

In [32]:
test_result = trainer.predict(test_dataset)
probs = test_result.predictions
preds = probs.argmax(axis=1)

In [33]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 81.25%


In [34]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_labels, preds)
print("MAE score:", mae)

MAE score: 0.25


In [35]:
save_directory = "/saved_models" 

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/saved_models/tokenizer_config.json',
 '/saved_models/special_tokens_map.json',
 '/saved_models/vocab.json',
 '/saved_models/merges.txt',
 '/saved_models/added_tokens.json')

In [36]:
tokenizer_fine_tuned = RobertaTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFRobertaForSequenceClassification.from_pretrained(save_directory)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /saved_models.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [43]:
# News from left

df['bias'][3]

2

In [44]:
predict_input = tokenizer_fine_tuned.encode(
    df['content'][3],
    truncation = True,
    padding = True,
    return_tensors = 'tf'    
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]
                                                      

prediction_value       # 0: Left, 1: Center, 2: Right

2